In [1]:
import tkinter as tk
import serial
import time
import threading
import winsound

In [ ]:
arduino = serial.Serial("COM3", 9600)
time.sleep(2)
#arduino.write(b'q')  # Asegurar que la bomba está apagada al iniciar

# Variables de control para el beep
beep_thread = None
running = False  # Controla si el beep debe seguir sonando
beep_activo = False  # Evita iniciar múltiples hilos del beep

# Función para ejecutar un beep en un hilo separado
def beep():
    global running
    running = True
    while running:
        winsound.Beep(1000, 300)  # Frecuencia: 1000 Hz, Duración: 300 ms
        time.sleep(0.3)  # Pequeña pausa para repetir el beep

# Iniciar beep en un hilo
def iniciar_beep():
    global beep_thread, beep_activo
    if not beep_activo:  # Solo iniciar si no está activo
        beep_activo = True
        beep_thread = threading.Thread(target=beep, daemon=True)
        beep_thread.start()

# Detener beep
def detener_beep():
    global running, beep_activo
    running = False
    beep_activo = False  # Marcar que el beep se ha detenido

# Función para encender el flujo de agua
def encender():
    arduino.write(b'a')
    label_estado.config(text="Estado: Encendida", fg="green")  # Cambiar texto en la interfaz

# Función para apagar el flujo de agua    
def apagar():
    arduino.write(b'q')
    label_estado.config(text="Estado: Apagada", fg="red")  # Cambiar texto en la interfaz
    
# Función para cambiar el color del círculo
def cambiar_color(color):
    canvas.itemconfig(circulo, fill=color)
    
# Función para leer datos del sensor en un hilo separado
def leer_sensor():
    global beep_activo  # Declaramos la variable como global para modificar su estado
    while True:
        try:
            data = arduino.readline().decode().strip()
        
            if data.isdigit():
                distancia = int(data)
                
                # Actualizar el texto de la etiqueta con la distancia
                label.config(text=f"Distancia: {distancia} cm")
                
                if distancia >= 13:
                    cambiar_color("red")  # Distancia segura, colorear rojo 
                    detener_beep()
                elif distancia >= 6:
                    cambiar_color("yellow")  # Distancia media, colorear de amarillo
                    detener_beep()
                elif distancia > 2:
                    cambiar_color("green")  # Distancia corta, colorear de verde
                    detener_beep()
                else:
                    cambiar_color("green")
                    iniciar_beep()  # Activar beep cuando la distancia es crítica
                    apagar()  # Apagar la bomba cuando el nivel es crítico
                    
        except Exception as e:
            print(f"Error: {e}")
        #time.sleep(0.1)
   
# Creación de la ventana principal
window = tk.Tk()
window.title("Control de la bomba de agua")
window.geometry("600x300")
window.configure(bg="lightgray") # Color de fondo


# Crear un lienzo para dibujar un círculo
canvas = tk.Canvas(window, width=100, height=100, bg="black", highlightthickness=0)
canvas.place(x=175, y=20)

# Dibujar el círculo inicial (rojo por defecto)
circulo = canvas.create_oval(10, 10, 90, 90, fill="red")

# Iniciar el hilo para leer datos del sensor y no bloquear la interfaz gráfica
threading.Thread(target=leer_sensor, daemon=True).start()

bttnEncender = tk.Button(window, text="Encender", command=encender, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttnEncender.place(x=100, y=200)

bttnApagar = tk.Button(window, text="Apagar", command=apagar, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttnApagar.place(x=250, y=200)

# Etiqueta para mostrar la distancia
label = tk.Label(window, text="Distancia: -- cm", bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
label.place(x=300, y=100)

label_estado = tk.Label(window, text="Estado: Apagada", bg="white", fg="red", font=("Arial", 12, "bold"), bd=3)
label_estado.place(x=300, y=130)

window.mainloop()

Error: name 'label' is not defined
Error: name 'label' is not defined
Error: 'utf-8' codec can't decode byte 0x85 in position 4: invalid start byte
